# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sklearn
import gensim
from gensim.models.keyedvectors import KeyedVectors
from collections import Counter

import tensorflow as tf
KERAS_BACKEND=tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM

import keras
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
tf.__version__

'2.3.0'

# Vinyasa Krama
What follows is my development of the proof-of-concept prototype for an app called Vinyasa Krama, an app which generates complete, well-structured, safe and compelling yoga sequences, via a bi-directional LSTM. In this app, the user is able to pick a desired 'peak' pose -- this is generally the most difficult posture located in the middle of a yoga class, and a custom class will be generated for them which will proper warm them up for and cool them down from that pose, properly preparing the relevant muslces and joints. The class is hence generated from the middle out. They can then take the class, and follow along with an animated yoga teacher (created in the Unity game engine) who will demonstrate the class pose by pose. 

# Loading In Yoga Class Data

Most of these DataFrames were scraped and created during my previous project, "Yoga Class-ification", with the exception of vinaysa_df and hatha_df, which I scraped to supplement my existent data to have a larger dataset. 

In [3]:
f = open("all_poses", "rb")
all_poses = pickle.load(f)
f.close()

f = open("flask_app_df", "rb")
poses_info = pickle.load(f)
f.close()

f = open("all_yoga_classes_df", "rb")
yoga_classes = pickle.load(f)
f.close()

f = open("poses_df", "rb")
class_poses = pickle.load(f)
f.close()

f = open("vinyasa_df", "rb")
more_vinyasa = pickle.load(f)
f.close()

f = open("hatha_df", "rb")
more_hatha = pickle.load(f)
f.close()

### Combining DataFrames and Excluding Restorative Class Types
I am excluding the classes that are of type "gentle", "restorative" and "yin", as these are very slow restorative/laying down type classes that I'm not aiming to create (yet) with this app. I want more dynamic, flow and exercise based classes to be generated, plus I have much more data availble in support of creating those types.

Furthermore, I want the LSTM to have rather consistent classes it's being trained on so it can best understand the sequences. 

In [4]:
df = pd.concat([yoga_classes, more_vinyasa, more_hatha])
df = df.loc[df["Class Type"].isin(["Vinyasa", "Hatha", "Power", "Iyengar", "Ashtanga"])]
df.reset_index(inplace=True)

In [5]:
df.head()

,index,Title,Poses,Class Type
0,0,←Slow Sunday Flow y Monday early,"[Easy Pose Hands To Heart, Easy Pose Hands Int...",Vinyasa
1,1,←Anahata,"[Mantra Section, Thunderbolt Pose, Easy Pose B...",Vinyasa
2,2,←CORE,"[Classic Sun Salutation Variation F, Chair Pos...",Vinyasa
3,3,←,"[Easy Pose, Easy Pose Warm Up Flow, Sun Saluta...",Vinyasa
4,4,←Vinyasa - Bench press & push up #,"[Corpse Pose, Corpse Pose Roll Under Spine, Wr...",Vinyasa


# Base Yoga Poses
I am concatenating together all of the yoga classes into one long list of lists to be fed to a neural net.

I am keeping these as lists, rather than one continuous string, because I want each individual pose to be a token -- if concatenating, NLP methods of tokenization would require I split on white space or some other arbitrary character, which would lose meaning. Instead, I am keeping each pose as an item inside a list, to preserve the meaning of the entire pose name (usually multiple words long) in each token. 

I am also reducing all of the 3600+ yoga pose variations down to their base poses. Many of the variations are not very distinct from one another, and for the sake of this MVP, the neural net will have richer context for fewer words seen more often, than thousands of different words -- it would not make sense to approach this problem that way, especially since the poses are in fact so similar. 

I, using industry knowledge, am adjusting from the 102 original base poses in my data, to include a few key variations which actually are distinct and represent different body movements. In this way I am not oversimplifying either. 

In [6]:
documents = [class_list for class_list in df["Poses"]]

In [7]:
base_poses = poses_info[["Pose Name", "Base Pose"]]

In [ ]:
# Commenting this out for readability but I manually looked at every pose in order to create the following...

# pd.set_option("display.max_rows", None)
# base_poses

In [8]:
changes = [([4, 17, 47, 50, 60], "Cycling Pose"),
    ([19, 40, 41, 44, 46, 28, 54, 67, 68, 72], "High Boat To Low Boat Flow"),
    ([64, 66, 69, 70], "Low Boat Pose"),
    ([86], "Staff Pose"), 
    ([87, 88, 117, 118], "Bound Angle Forward Bend"), 
    ([125, 135, 136, 138, 140, 141], "One Legged Bow Pose"), 
    ([216, 217, 218, 219], "Pigeon Pose"), 
    ([233, 234, 235, 236, 237, 238, 239], "Thunderbolt Pose"), 
    ([248, 269, 270, 271, 274], "Cow Pose"), 
    ([259, 260, 261, 262, 263, 364, 265, 266, 267, 268, 272, 273, 275], "Cat Pose"),
    ([291], "Hurdlers Pose"),
    ([293, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 372], "Revolved Chair Pose"), 
    ([300, 303, 306, 307, 308], "Chair Pose With Airplane Arms"),
    ([365, 369, 370], "Figure Four Pose"), 
    ([368], "Shiva Squat Pose"), 
    ([520, 521], "Supine Spinal Twist Pose"), 
    ([587, 588], "Flying Pigeon Pose"), 
    ([594, 589], "Side Crow Pose"), 
    ([599], "Baby Crow Pose"), 
    ([663, 664, 665, 666, 667, 668], "One Legged Mountain Pose"), 
    ([687, 758], "One Handed Downward Facing Dog Pose"),
    ([688, 689, 690, 759, 760], "Standing Splits Pose"), 
    ([691, 692, 693, 694, 724, 735, 736, 737, 761, 762, 763, 764], "Three Legged Downward Facing Dog Pose"), 
    ([700, 717], "Downward Facing Dog Upward Facing Dog Pose Flow"), 
    ([714], "Downward Facing Dog Pose Plank Pose Flow"), 
    ([715], "Downward Facing Dog Pose Table Top Pose Flow"), 
    ([747], "Downward Facing Dog Pose Knee To Nose"),
    ([748], "Downward Facing Dog Pose Shoulder Taps"), 
    ([749], "Downward Facing Dog Pose to Low Lunge Pose Flow"), 
    ([865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901], "Easy Pose"), 
    ([924, 925, 926], ""), # this was "cactus arms", its a variation that won't make sense in most contexts. intentionally deleting this and will filter out empty strings upon generation. 
    ([998], "Half Lotus Pose"), 
    ([999], "Lotus Pose"), 
    ([1016, 1017], "Eye Exercise"), ([1055, 1056, 1057, 1058, 1061], "Scorpion Pose"), 
    ([1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151], "Chaturanga Dandasana"),
    ([1387], "Headstand Pose Eagle Legs"), 
    ([1390], "Headstand Pose Lotus Legs"), 
    ([1392], "Headstand Pose Wide Legs"), 
    ([1395, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407], "Tripod Headstand Pose"), 
    ([1464, 1465, 1479, 1480, 1481, 1482], "Revolved High Lunge Pose"), 
    ([1466, 1467, 1468, 1469, 1483], "Runners Lunge Pose"), 
    ([1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533], "Wide Legged Forward Fold"), 
    ([1487, 1509, 1510, 1511, 1512], "Revolved Wide Legged Forward Fold"),
    ([1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498], "Wide Legged Forward Fold With Halfway Lift"),
    ([1489, 1490, 1527], "Five Pointed Star Pose"), 
    ([1508], "Pyramid Pose"), 
    ([1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566], "Pyramid Pose"), 
    ([1584], "Flying Lizard Pose"), 
    ([1673, 1684, 1685, 1591, 1700, 1708, 1710, 1711, 1712], "Revolved Low Lunge Pose"),
    ([1732, 1733, 1745, 1758, 1758, 1776, 1777, 1778, 1784, 1790, 1791, 1791], "One Legged Moutain Pose"), 
    ([1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852], "Easy Pose"), 
    ([1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882], "Standing Side Bend Pose"), 
    ([1891, 1892, 1893, 1901, 1901, 1903, 1904, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922], "King Pigeon Pose"), 
    ([1927, 1928, 1932], "Grasshopper Pose"), 
    ([1929], "Dragonfly Pose"), 
    ([1930], "Eight Angle Pose"),
    ([1934, 1970, 1971, 1972], "One Legged Plank Pose"), ([1939], "Side Plank Pose"), 
    ([1958, 1959, 1960, 1961, 1967, 1968, 1969], "Forearm Plank Pose"), 
    ([2173], "Lotus Pose"), 
    ([2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200], "Revolved Extended Side Angle Pose"), 
    ([2189, 2195, 2196], "Revolved High Lunge Pose"), 
    ([2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368], "Skandasana"),
    ([2390, 2397], "Anantasana"), 
    ([2415, 2416, 2417, 2418, 2419, 2420, 2443], "Visvamitrasana Pose"), 
    ([2422, 2427, 2428, 2430, 2431, 2433, 2453, 2458, 2460], "Side Plank Pose With Leg Variation"), 
    ([2436, 2437, 2438, 2439, 2440, 2441], "Forearm Side Plank Pose"), 
    ([2465, 2466], "Wild Thing Pose"), 
    ([2482, 2483, 2484, 2486, 2487, 2488, 2489, 2480, 2492, 2493], "Half Splits Pose"), 
    ([2494], "Standing Splits Pose"), 
    ([2592, 2593, 2594], "Firefly Pose"), 
    ([2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621], "Forward Fold Pose With Halfway Lift"), 
    ([2688, 2689, 2692, 2717], "Bird Of Paradise Pose"), 
    ([2700, 2701, 2703], "Revolved Hand To Big Toe Pose"), 
    ([2750, 2758], "Half Sun Salutation"), 
    ([2759], "Second Half Of Sun Salutation"), 
    ([2771, 2772], "Banana Pose"), 
    ([2795], "Supine Spinal Twist Pose"), 
    ([2797, 2798, 2833, 2848, 2849, 2850], "Tiger Pose"), 
    ([2799, 2816, 2827], "Table Top Knee To Nose Flow"), 
    ([2800, 2817], "Child Pose Table Top Pose Flow"), 
    ([2801, 2831, 2834, 2837, 2840, 2841, 2842], "Table Top Pose With One Leg Extended Back"), 
    ([2809, 2810, 2811, 2812, 2813, 2814, 2815], "Revolved Table Top Pose"), 
    ([2852, 2853, 2854, 2855], "Table Top Balancing Pose, Opposite Arm and Leg Extended"), 
    ([2968, 2972, 2983, 2974, 2975, 2976, 2977, 3005, 3006, 3007, 3008], "Revolved Triangle Pose"), 
    ([3023, 3024, 3025, 3026, 3031, 3032, 3038, 3039, 3040], "Reverse Table Top Pose"), 
    ([3037], "Flip The Dog Pose"), 
    ([3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063], "Anantasana"), 
    ([3064, 3065, 3066, 3067, 3070, 3071, 3072, 3073, 3074, 3075, 3079], "Warrior Pose I"), 
    ([3068, 3069, 3076, 3077, 3078], "Humble Warrior Pose"), 
    ([3106, 3107, 3108, 3109, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121], "Warrior Pose II"), 
    ([3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144], "Warrior Pose III"), 
    ([3134], "Shiva Squat Pose"), 
    ([3131, 3132, 3133], "Airplane Pose"), 
    ([3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190], "Half Wind Release Pose")]

In [9]:
for i in changes:
    base_poses.loc[i[0], "Base Pose"] = i[1]

In [10]:
len(base_poses["Base Pose"].unique())

170

I know have 170 distinct poses that I will be feeding to my neural net. Now creating a list of class data to feed to the neural net out of the base documents only:  

In [11]:
base_documents = []
for i in range(len(documents)):
    temp_df = pd.DataFrame(documents[i], columns=["0"])
    temp_df = pd.merge(temp_df, base_poses, how="left", left_on="0", right_on="Pose Name")
    temp_df = temp_df.dropna(how="any")
    new_doc = [pose for pose in temp_df["Base Pose"]]
    base_documents.append(new_doc)

# Determining Start and End Poses
For the sake of class generation, since generation is starting with the user's desired most difficult pose in the middle of the class, I want the classes to not be generated to some arbitary pre-set length, but instead continue generating until converging to the natural start and end yoga poses of a class. Hence I am finding the most common start and end poses across all classes. 

In [25]:
first_poses = []
for doc in base_documents:
    if doc:
        first_poses.append(doc[0])

In [24]:
last_poses = []
for doc in base_documents:
    if doc:
        last_poses.append(doc[-1])

In [26]:
for i in range(10):
    print(list(Counter(first_poses).keys())[i], ":", list(Counter(first_poses).values())[i])

Easy Pose : 12182
Thunderbolt Pose : 743
Sun Salutation : 1764
Corpse Pose : 5272
Table Top Pose : 669
Lotus Pose : 433
Pranayama : 2099
Reclined Bound Angle Pose : 1207
Child Pose : 2534
Cycling Pose : 326


In [27]:
for i in range(10):
    print(list(Counter(last_poses).keys())[i], ":", list(Counter(last_poses).values())[i])

Corpse Pose : 20486
Side Lying Corpse Pose : 637
Downward Facing Dog Pose : 684
Thunderbolt Pose : 193
Half Sun Salutation : 49
Thread The Needle Pose : 117
Revolved Wide Legged Forward Fold : 32
Bridge Pose : 630
Pranayama : 971
Chair Pose : 280


The vast majority of classes begin with easy pose (simple cross-legged seat, often for meditation). Pranayama which is also very high just means "breathing", which is not itself a "pose", but a breath technique which would commonly practiced while in sitting meditation (easy pose). By far the most common last pose is 'corpse pose', or final savasana, lying down meditation. 

Without even having done the above exploration, I would have been inclined to start the classes with 'easy pose' and end with 'corpse' pose just due to my personal experience with yoga, but now I have the empirical evidence to support that decision. 

# Word2Vec Embeddings
I am using a custom word embedding model to find similarities between poses. I will then implement transfer learning, using this pre-trained word embedding model as my initial weights for the LSTM. 

In [28]:
embeddings = gensim.models.Word2Vec(base_documents, size=100, window=5, min_count=1, sg=1)

In [38]:
embeddings_size = len(list(embeddings.wv.vocab.items()))
embeddings_size

169

## Analyzing Embedding Accuracy

In [32]:
print(embeddings.similarity("Corpse Pose", "Dancer Pose"))
print(embeddings.similarity("Corpse Pose", "Extended Side Angle Pose"))
print(embeddings.similarity("Corpse Pose", "Child Pose"))
print(embeddings.similarity("Corpse Pose", "Wind Release Pose"))

0.098134644
0.1699742
0.39498302
0.81561786


The above looks very accurate. When compared to 'corpse pose', which is lying down meditation, model indicates very little similarity to difficulty balancing and strength-heavy standing poses, mild similarity to an intense prone stretch pose, moderate similarity to a relaxation prone pose, and very high similarity to wind release pose, which often tends to be practiced immediately before corpse pose, and is also very easy and practiced while laying on the back. 


In [35]:
def analogy(worda, wordb, wordc):
    result = embeddings.most_similar(negative=[worda], positive=[wordb, wordc])
    return result[0][0]

In [36]:
analogy("Downward Facing Dog Pose", "Three Legged Downward Facing Dog Pose", "Table Top Pose")

'Table Top Balancing Pose, Opposite Arm and Leg Extended'

In [39]:
analogy("Warrior Pose II", "Extended Side Angle Pose", "Mountain Pose")

'Palm Tree Pose'

In [42]:
analogy("Revolved Triangle Pose", "Triangle Pose", "Revolved Chair Pose")

'Chair Pose'

The above analogies also indicate that the embeddings model has picked up a good deal of nuanced meaning about the poses. When down dog to a balancing version of downdog is compared to table top, it offers a balancing version of table top. When an arms extended version of warrior II is compared to mountain pose, it offers an arms extended version of mountain (palm tree). When a revolved version of triangle and regular triangle is compared to a revolved version of chair, it offers regular chair. Wonderful! 

Saving the embeddings as a matrix for use in the neural net: 

In [43]:
embedding_matrix = np.zeros((len(embeddings.wv.vocab)+1, 100))
for i in range(len(embeddings.wv.vocab)):
    embedding_vector = embeddings.wv[embeddings.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [44]:
embedding_dim = embeddings.vector_size

# Bi-Directional LSTM Neural Network

## Tokenization


In [46]:
tokens = [pose for yogaclass in base_documents for pose in yogaclass]
print("Total tokens: ", len(tokens))
print("Unique tokens: ", len(set(tokens)))

Total tokens:  1624681
Unique tokens:  169


## Input Sequences
I am training the network on an input sequence length of 3 poses, where it will then learn the expected fourth pose. This small input sequence length size is ideal for seeing all the poses in many many contexts, and allows for easy generation from a selected single base pose. 

In [47]:
length = 3 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    sequences.append(seq)
print("Total Sequences:", len(sequences))

Total Sequences: 1624677


In [48]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
token_sequences = tokenizer.texts_to_sequences(sequences)
vocab_size = len(tokenizer.word_index) + 1
print("Vocab Size:", vocab_size)

Vocab Size: 170


In [49]:
X = np.asarray([np.asarray(x[:-1]) for x in token_sequences])
y = [y[-1] for y in token_sequences]
y = to_categorical(y, num_classes=vocab_size)
seq_length=len(X[1])

## Neural Net Architecture 
My choice of deep learning model is a bi-directional LSTM. This is the ideal choice for my purposes. 

Long Short-Term Memory NNs are considered the best performing models for sequence prediction, and are excellent for text generation -- and I am, in this project, doing both. I am generating text that is a sequence of yoga poses that should go in a fairly specific order in order to be safe and comprise a good class. 

A bi-directional LSTM is an even more ideal choice -- Since I am generating my classes from the middle (peak pose) out, I essentially need to predict and generate half of the from the peak to the end in a forward direction, and the other half of the sequence from the peak to the beginning in a backwards direction. A bi-directional LSTM is uniquely poised to allow me to do this -- by providing the input sequence to each layer twice, once as is and once reversed, and then concatenating the outputs -- the msdel has twice as much context about each pose, and is then able to predict what pose would *precede* a given sequence, as well as follow it. 

I have chosen after some experimentation to use 3 hidden bidirectional LSTM layers, with dropout layers between each, of a size roughly comparable to my vocabulary. I then have 2 activation layers, the first using RELU and the second softmax as any form of sequence/text prediction is categorized as a multi-class classification problem. 

I train the model with a modest batch size of 128 (given the 1.6mil input sequences), over 100 epochs, using the adam optimizer. 

In [50]:
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    model.add(Dropout(0.05))
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    model.add(Dropout(0.05))
    model.add(Bidirectional(LSTM(200)))
    model.add(Dropout(0.05))
    model.add(Dense(100, activation="relu"))
    model.add(Dense(vocab_size, activation="softmax"))
    print(model.summary())
    return model

In [51]:
def fit_model(model, X, y):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, batch_size=128, epochs=100, verbose=10)

In [53]:
lstm_model = create_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         17000     
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 400)         481600    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 400)         0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 400)         961600    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 400)         0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 400)               961600    
_________________________________________________________________
dropout_5 (Dropout)          (None, 400)              

In [185]:
fit_model(lstm_model, X, y)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [55]:
lstm_model.save('lstm_model.h5')

In [57]:
lstm_model = load_model('lstm_model.h5', compile=False)

# Yoga Class Generation 

In [58]:
def generate_class(model, tokenizer, word_embedding, peak_pose, stop_pose, max_length):

    # generate seed text of 3 poses (as LSTM was trained on) from the input desired peak pose, by randomly selecting two of the most similar poses to the peak. 
    seed_text = [peak_pose, embeddings.most_similar(peak_pose, topn=10)[np.random.choice(range(10))][0], embeddings.most_similar(peak_pose, topn=10)[np.random.choice(range(10))][0]]
    in_text = seed_text

    # create yoga class, and make sure it includes the user's desired peak pose
    yoga_class = list()
    first_half = list()
    second_half = list()
    yoga_class.append(peak_pose.lower())

    while True:
        encoded = tokenizer.texts_to_sequences([in_text])

        # select next pose based on models probability distribution
        prediction_output = model.predict(encoded)
        yhat = np.random.choice(len(prediction_output[0]), p=prediction_output[0])
        # argsort version:
        # predictions = np.argsort(model.predict(encoded), axis=-1)[-10:][::-1]
        # yhat = np.random.choice(predictions[0])

        out_word = ""
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        
        # append pose to current class, and update input text
        # also add clarification text for user to repeat the pose on the other side (if its an imbalanced pose) if given two of the same pose in a row
        if out_word != "":
            if out_word == yoga_class[-1]:
                in_text.append(out_word)
                out_word += ", repeat other side"
                yoga_class.append(out_word)
            else: 
                yoga_class.append(out_word)
                in_text.append(out_word)
        if out_word == stop_pose:
            break

        # if sequence gets too long without converging to a natural ending, start over. 
        if len(yoga_class) == max_length:
                in_text = seed_text
                yoga_class = [peak_pose.lower()]
    return yoga_class

In [615]:
possible_peak_poses = ['Center Splits Pose', 'Hurdlers Pose', 'Revolved Chair Pose', 'Figure Four Pose', 'Crane Pose', 'Flying Pigeon Pose', 'Side Crow Pose', 'Baby Crow Pose', 'Crow Pose', 'Dancer Pose', 'Eagle Pose', 'Feathered Peacock Pose', 'Scorpion Pose', 'Flamingo Pose', 'Foot Behind The Head Pose', 'Half Moon Pose', 'Handstand Pose', 'Headstand Pose', 'Headstand Pose Eagle Legs', 'Headstand Pose Lotus Legs', 'Headstand Pose Wide Legs', 'Tripod Headstand Pose', 'Revolved High Lunge Pose', 'Pyramid Pose', 'Flying Lizard Pose', 'Palm Tree Pose', 'King Pigeon Pose', 'Grasshopper Pose', 'Dragonfly Pose', 'Eight Angle Pose', 'Revolved Extended Side Angle Pose', 'Visvamitrasana Pose', 'Wild Thing Pose', 'Splits Pose', 'Firefly Pose', 'Bird Of Paradise Pose', 'Standing Hand To Big Toe Pose', 'Revolved Hand To Big Toe Pose', 'Tree Pose', 'Revolved Triangle Pose', 'Triangle Pose', 'Flip The Dog Pose', 'Warrior Pose I', 'Humble Warrior Pose', 'Warrior Pose II', 'Warrior Pose III', 'Airplane Pose', 'Wheel Pose']

In [616]:
f = open("peak_poses.pkl", "wb")
pickle.dump(possible_peak_poses, f)
f.close()

In [699]:
peak_pose = np.random.choice(possible_peak_poses)
peak_pose = 'Flying Pigeon Pose'

In [477]:
for i in range(100):
    peak_pose = np.random.choice(possible_peak_poses)
    second_half = generate_class(forward_model, tokenizer, embeddings, peak_pose, 40)
    print(second_half)
    print("\n")

['humble warrior pose', 'child pose', 'downward facing dog pose', 'mountain pose', 'half lord of the fishes pose', 'standing forward fold pose', 'cycling pose', 'seated forward bend pose', 'eagle pose', 'mountain pose', 'mountain pose, repeat other side', 'thunderbolt pose', 'thunderbolt pose, repeat other side', 'child pose', 'child pose, repeat other side', 'lotus pose', 'bound angle pose', 'half lord of the fishes pose', 'corpse pose']


['palm tree pose', 'mountain pose', 'palm tree pose', 'palm tree pose, repeat other side', 'shoulder stretches', 'standing twists', 'chair pose', 'chair pose, repeat other side', 'head to knee pose', 'thunderbolt pose', 'torso stretch pose', 'easy pose', 'easy pose, repeat other side', 'half lord of the fishes pose', 'torso stretch pose', 'seated straddle pose', 'happy baby pose', 'reclined big toe pose', 'bridge pose', 'half lord of the fishes pose', 'pranayama', 'pranayama, repeat other side', 'sun salutation', 'standing forward fold pose', 'corps

KeyboardInterrupt: 

In [705]:
first_half = generate_class(forward_model, tokenizer, embeddings, peak_pose, "easy pose", 40)[::-1]

In [706]:
second_half = generate_class(forward_model, tokenizer, embeddings, peak_pose, "corpse pose", 40)

In [707]:
class_length = len(first_half) + len(second_half)
class_length

47

In [708]:
yoga_class = first_half + second_half

In [709]:
yoga_class

['easy pose',
 'cat cow pose',
 'wind release pose',
 'plough pose',
 'happy baby pose',
 'corpse pose',
 'legs up the wall pose',
 'eye of the needle pose',
 'half lord of the fishes pose',
 'eye of the needle pose',
 'happy baby pose',
 'reclined big toe pose',
 'supine spinal twist pose, repeat other side',
 'supine spinal twist pose',
 'bridge pose, repeat other side',
 'bridge pose',
 'garland pose',
 'seated forward bend pose, repeat other side',
 'seated forward bend pose',
 'bound angle pose, repeat other side',
 'bound angle pose',
 'extended side angle pose',
 'flamingo pose, repeat other side',
 'flamingo pose',
 'crane pose, repeat other side',
 'crane pose',
 'baby crow pose',
 'flying pigeon pose',
 'flying pigeon pose',
 'wind release pose',
 'lotus pose',
 'happy baby pose',
 'child pose',
 'half lord of the fishes pose',
 'plank pose',
 'seated straddle pose',
 'bound angle pose',
 'half lord of the fishes pose',
 'bow pose',
 'cobra pose',
 'cow face pose',
 'pigeon p

In [ ]:
def generate_class_2(model, tokenizer, word_embedding, peak_pose, stop_pose, max_length):

    # generate seed text of 3 poses (as LSTM was trained on) from the input desired peak pose, by randomly selecting two of the most similar poses to the peak. 
    seed_text = [peak_pose, embeddings.most_similar(peak_pose, topn=10)[np.random.choice(range(10))][0], embeddings.most_similar(peak_pose, topn=10)[np.random.choice(range(10))][0]]
    in_text = seed_text

    # create yoga class, and make sure it includes the user's desired peak pose
    yoga_class = list()
    yoga_class.append(peak_pose.lower())

    for i in range(max_length):
        if not i % 2:
            encoded = tokenizer.texts_to_sequences([in_text[::-1]])

            # select next pose based on models probability distribution
            prediction_output = model.predict(encoded)
            yhat = np.random.choice(len(prediction_output[0]), p=prediction_output[0])
    
            out_word = ""
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            
            if out_word != "":
                if out_word == yoga_class[0]:
                    

            # append pose to current class, and update input text
            # also add clarification text for user to repeat the pose on the other side (if its an imbalanced pose) if given two of the same pose in a row
            if out_word != "":
                if out_word == yoga_class[-1]:
                    in_text.append(out_word)
                    out_word += ", repeat other side"
                    yoga_class.append(out_word)
                else: 
                    yoga_class.append(out_word)
                    in_text.append(out_word)
            if out_word == stop_pose:
                break

        # if sequence gets too long without converging to a natural ending, start over. 
        if len(yoga_class) == max_length:
                in_text = seed_text
                yoga_class = [peak_pose.lower()]
    return yoga_class

In [ ]:
def generate_class_2(model, tokenizer, word_embedding, peak_pose, stop_pose, max_length):

    # generate seed text of 3 poses (as LSTM was trained on) from the input desired peak pose, by randomly selecting two of the most similar poses to the peak. 
    seed_text = [peak_pose, embeddings.most_similar(peak_pose, topn=10)[np.random.choice(range(10))][0], embeddings.most_similar(peak_pose, topn=10)[np.random.choice(range(10))][0]]
    in_text = seed_text

    # create yoga class, and make sure it includes the user's desired peak pose
    yoga_class = list()
    yoga_class.append(peak_pose.lower())

    for i in range(2):
        while True:
            encoded = tokenizer.texts_to_sequences([in_text[::-1]])

            # select next pose based on models probability distribution
            prediction_output = model.predict(encoded)
            yhat = np.random.choice(len(prediction_output[0]), p=prediction_output[0])

            out_word = ""
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            
            if out_word != "":
                if out_word == yoga_class[0]:
                    

            # append pose to current class, and update input text
            # also add clarification text for user to repeat the pose on the other side (if its an imbalanced pose) if given two of the same pose in a row
            if out_word != "":
                if out_word == yoga_class[-1]:
                    in_text.append(out_word)
                    out_word += ", repeat other side"
                    yoga_class.append(out_word)
                else: 
                    yoga_class.append(out_word)
                    in_text.append(out_word)
            if out_word == stop_pose:
                break

        # if sequence gets too long without converging to a natural ending, start over. 
        if len(yoga_class) == max_length:
                in_text = seed_text
                yoga_class = [peak_pose.lower()]
    return yoga_class

# Pickling
For use in flask app. 

In [ ]:
f = open("word_embeddings.pkl", "wb")
pickle.dump(embeddings, f)
f.close()

f = open("tokenizer.pkl", "wb")
pickle.dump(tokenizer, f)
f.close